# Clustering Crypto

In [4]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [17]:
file_path = 'Resources/crypto_data.csv'
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [18]:
# Keep all the cryptocurrencies that are being traded.
currencies_df = crypto_df[crypto_df.IsTrading == True]
currencies_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [38]:
# Keep all the cryptocurrencies that have a working algorithm.
working_algorithm_df = currencies_df.dropna(subset=["Algorithm"], axis=0)
working_algorithm_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [46]:
# Remove the "IsTrading" column. 
clean_crypto_df = working_algorithm_df.drop(["IsTrading"], axis = 1)
clean_crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [49]:
# Remove rows that have at least 1 null value.
clean_crypto_df = clean_crypto_df.dropna()
clean_crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [51]:
# Keep the rows where coins are mined.
new_crypto_df = clean_crypto_df.loc[clean_crypto_df["TotalCoinsMined"] > 0]
new_crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [61]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names_df = pd.DataFrame(new_crypto_df["CoinName"])
names_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [70]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = new_crypto_df.drop(["CoinName"], axis = 1)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [72]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns = ["Algorithm", "ProofType"])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Deliverable 2: Reducing Data Dimensions Using PCA

In [74]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.33690156,  1.04469578, -0.62947665],
       [-0.32026876,  1.04479594, -0.62976815],
       [ 2.30948465,  1.61413565, -0.65432638],
       ...,
       [ 0.32630486, -2.30670683,  0.42558661],
       [-0.14287409, -2.06954564,  0.43544015],
       [-0.30203845,  0.866912  , -0.32439234]])

In [81]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data=X_pca, columns=["PC 1", "PC 2", "PC 3" ], index= X.index
)
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.336902,1.044696,-0.629477
404,-0.320269,1.044796,-0.629768
1337,2.309485,1.614136,-0.654326
BTC,-0.138517,-1.304552,0.223040
ETH,-0.155362,-1.965754,0.418310


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [83]:
# Create an elbow curve to find the best value for K.

inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve") 

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [86]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,

In [89]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = names_df["CoinName"] 

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.336902,1.044696,-0.629477,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.320269,1.044796,-0.629768,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.309485,1.614136,-0.654326,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.138517,-1.304552,0.223040,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.155362,-1.965754,0.418310,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.170130,-1.087818,-0.044509,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.390538,1.141413,-0.456789,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.156476,-2.256307,0.458176,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.153806,-1.965856,0.418293,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.142873,-2.069546,0.435440,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [100]:
# Creating a 3D-Scatter with the PCA data and the clusters

fig = px.scatter_3d(
    clustered_df,
    x ="PC 1",
    y ="PC 2",
    z ="PC 3",
    color ="class",
    symbol ="class",
    width =800,
    hover_name = "CoinName",
    hover_data = ["Algorithm"]
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [101]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table()

:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class]

In [115]:
# Print the total number of tradable cryptocurrencies.
print(f' There are {clustered_df["CoinName"].count()} tradable cryptocurrencies')

 There are 532 tradable cryptocurrencies


In [120]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaled_df_copy = clustered_df.copy()
scaled_df = scaled_df_copy.drop(["Algorithm", "ProofType", "PC 1", "PC 2", "PC 3", "CoinName", "class"], axis=1)

minmax = MinMaxScaler().fit_transform(scaled_df)
minmax

array([[0.00000000e+00, 4.20000000e-11],
       [1.06585544e-03, 5.32000000e-04],
       [2.95755135e-02, 3.14159265e-01],
       ...,
       [9.90135079e-04, 1.40022261e-03],
       [7.37028150e-06, 2.10000000e-05],
       [1.29582282e-07, 1.00000000e-06]])

In [126]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.

plot_df = pd.DataFrame(data=minmax, columns = ["TotalCoinSupply", "TotalCoinsMined"], index = clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.

plot_df["CoinName"] = names_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 

plot_df["class"] = clustered_df["class"]

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,class
42,0.000000,4.200000e-11,42 Coin,0
404,0.001066,5.320000e-04,404Coin,0
1337,0.029576,3.141593e-01,EliteCoin,0
BTC,0.000018,2.100000e-05,Bitcoin,1
ETH,0.000109,0.000000e+00,Ethereum,1
LTC,0.000064,8.400000e-05,Litecoin,1
DASH,0.000009,2.200000e-05,Dash,0
XMR,0.000017,0.000000e+00,Monero,1
ETC,0.000115,2.100000e-04,Ethereum Classic,1
ZEC,0.000007,2.100000e-05,ZCash,1


In [127]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".

plot_df.hvplot.scatter(
    x= "TotalCoinsMined",
    y= "TotalCoinSupply",
    hover_cols=["CoinName"],
    by="class",
)

:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)